In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import hydro_serving_grpc as hs_grpc
import hydro_serving_grpc.manager as hsm
from client import HydroServingClient, HydroServingServable, HydroServingModel

In [3]:
hs_client = HydroServingClient('localhost:9090')

In [9]:
hs_model = hs_client.get_model('adult-columnar')
hs_model._HydroServingModel__id

4

In [20]:
x = pd.DataFrame(np.random.randint(-1000, 1000, 5*12).reshape(5, 12), columns=hss.contract.input_names)
y = hs_model(x)
print(y)

NameError: name 'hss' is not defined

In [13]:
servable = hs_client.deploy_servable('adult-columnar')
servable

2019-07-10 11:07:28.934 | INFO     | client:deploy_servable:78 - adult-columnar-1-huge-rain is ServableStatus.STARTING
2019-07-10 11:07:31.700 | INFO     | client:deploy_servable:78 - adult-columnar-1-huge-rain is ServableStatus.SERVING


adult-columnar v1, servable_id = adult-columnar-1-huge-rain

In [18]:
x = pd.DataFrame(np.random.randint(-1000, 1000, 5*12).reshape(5, 12), columns=servable.contract.input_names)
y = servable(x, _profile=True)
print(y)

{'Prediction': array([[0],
       [1],
       [1],
       [0],
       [1]])}


In [23]:
from hydro_serving_grpc.reqstore import reqstore_client
rs_client = reqstore_client.ReqstoreClient("127.0.0.1:9090", insecure=True)

entries = []
for r in rs_client.subsampling("4", amount=100):
    entries.extend(r.entries)
    
# print(max([len(e.request.inputs['Age'].int64_val)for e in entries]))
requests = [servable.contract.decode_request(e.request) for e in entries]

In [25]:
requests

[{'Sex': array([[1]]),
  'Relationship': array([[1]]),
  'Capital Gain': array([[1]]),
  'Occupation': array([[1]]),
  'Race': array([[1]]),
  'Capital Loss': array([[1]]),
  'Country': array([[1]]),
  'Hours per week': array([[1]]),
  'Workclass': array([[1]]),
  'Marital Status': array([[1]]),
  'Age': array([[1]]),
  'Education': array([[1]])},
 {'Education': array([[1]]),
  'Sex': array([[1]]),
  'Relationship': array([[1]]),
  'Capital Gain': array([[1]]),
  'Occupation': array([[1]]),
  'Race': array([[1]]),
  'Capital Loss': array([[1]]),
  'Country': array([[1]]),
  'Hours per week': array([[1]]),
  'Workclass': array([[1]]),
  'Marital Status': array([[1]]),
  'Age': array([[1]])},
 {'Sex': array([[1]]),
  'Relationship': array([[1]]),
  'Capital Gain': array([[1]]),
  'Occupation': array([[1]]),
  'Race': array([[1]]),
  'Capital Loss': array([[1]]),
  'Country': array([[1]]),
  'Hours per week': array([[1]]),
  'Workclass': array([[1]]),
  'Marital Status': array([[1]]),
  '

In [297]:
# concat requsests
rs = []
order = hss.contract.input_names
for r in requests:
    columns = []
    for feature_name in order:
        columns.append( r[feature_name])
    ndarray = np.hstack(columns)
    df = pd.DataFrame(ndarray, columns=order)
    rs.append(df)
data = pd.concat(rs)

In [298]:
data

,Age,Workclass,Education,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country
0,1,1,1,15050,1,1,1,1,100,1,1,1
1,23423,32432,324,234,23423,23423,23423,23423,23423,23432,32432,234234
0,48,-678,-761,225,-21,870,-223,764,693,366,-619,-95
1,515,-643,976,20,-476,-478,-711,-796,-587,848,-88,-685
2,-782,-963,-537,904,-295,-903,-409,733,-247,950,-629,450
3,801,-855,935,-25,24,-877,-787,-366,-138,363,801,892
4,792,859,213,601,511,-804,-781,-102,153,518,350,745
0,-300,-983,611,885,623,820,891,598,-882,898,-763,532
1,-845,-794,422,550,533,-61,-288,846,-866,809,-329,-430
2,-554,212,-160,87,-245,-253,686,-265,143,302,639,-620


In [293]:
import grpc 

import hydro_serving_grpc as hs 

channel = grpc.insecure_channel("localhost:9090") 

stub = hs.PredictionServiceStub(channel) 

model_spec = hs.ModelSpec(name="big_kek", signature_name="predict")

for i in range(20):
    x = pd.DataFrame(np.random.randint(-1000, 1000, 5*12).reshape(5, 12), columns=hss.contract.input_names)

    proto_x = hs_model.contract.make_proto(hs_model.contract.make_input_dict(x, {}))

    request = hs.PredictRequest(model_spec=model_spec, inputs=proto_x) 


    result = stub.Predict(request)
    result